# Streamflow Drought Statistical Suite
## v1, 2024

The suite of statistical metrics within this notebook align with the metrics found in [Simeone et al., 2024; Table 1](https://doi.org/10.3390/w16202996) and are geared toward the evaluation of streamflow drought in particular. Within this notebook are custom-defined Python functions to compare modeled/simulated steramflow data against observational streamflow data over time. The code contained within the notebook is adapted from the original functions found in the R package _HyMED_ [(Simeone and Foks, 2024)](https://doi.org/10.5066/P1TLARC5).

This notebook will briefly describe each metric and contain the code to calculate each metric.  This notebook can be sourced into analysis notebooks to retrieve access to these functions without having to copy and paste the metric of interest. 

##### Streamflow drought statistical metrics for daily streamflow drought evaluation for different metric categories. Additional calculation details can be found in Text S1 [Simeone et al., 2024; Table 1](https://doi.org/10.3390/w16202996).

| Category	| Statistic	| Description	| Units | Range (Perfect)	| Comments |
| :-----    | :-----    |:-----         |:-----   | :-----    | :----
| Event Classification	| Cohen’s kappa |	Cohen’s kappa statistic for inter-rater reliability,  measure of agreement between categorical variables | unitless |	−1 to 1 (1)	| A measure of agreement relative to the probability of achieving results by chance. |
|Error Components | Spearman’s r | Spearman’s rank correlation coefficient | unitless | −1 to 1 (1) | A nonparametric estimator of correlation for flow timing. |
|Error Components | Ratio of standard deviations | Ratio of simulated to observed standard deviations (for the scorecard this is presented as the absolute deviation from the target of 1)	| unitless | 0 to Inf (1)	| Indicates if the flow variability is being over or underestimated. |
|Error Components | Percent bias | Percent bias (simulated minus observed) (for the scorecard this is presented as the absolute percent bias) | percent | −100 to Inf (0) |   Indicates if total streamflow volume is being over or underestimated. |
| Drought Signatures | Drought Duration | Normalized mean absolute error (NMAE) in the annual time series of drought duration (units in time or days), this is the sum of days of drought each year for a given threshold. |	time (days) | 0 to Inf (0) |	Indicates how well the model simulates annual drought durations. |
| Drought Signatures | Drought Intensity | NMAE in annual time series of the distance the minimum percentile is below the drought threshold, this is the overall maximum distance below the threshold for any drought during the year (minimum percentile). | percentile | 0 to Inf (0) | Indicates how well the model simulates annual minimum flow.
| Drought Signatures | Drought Severity (Flow Deficit Volume) | NMAE in the annual time series of drought deficit volume, this is the sum of drought deficits for all droughts during the year. | flow units (cms or cfs) - days |	0 to Inf (0) | Indicates how well the model simulates annual flow deficit. This is a measure of drought severity. |

### References:
- Simeone, C.E., and Foks, S.S. (2024). HyMED—Hydrologic Model Evaluation for Drought: R package version 1.0.0, U.S. Geological Survey software release, https://doi.org/10.5066/P1TLARC5.
- Simeone, C., Foks, S., Towler, E., Hodson, T., & Over, T. (2024). Evaluating Hydrologic Model Performance for Characterizing Streamflow Drought in the Conterminous United States. Water, 16(20), 2996. https://doi.org/10.3390/w16202996.

## Import libraries

In [ ]:
import logging
import numpy as np
import pandas as pd
from sklearn.metrics import cohen_kappa_score
from scipy.stats import spearmanr

## Percentile and Threshold Calculations
This function is adapted from _calculate_percentiles_single.R_ in HyMED [(Simeone and Foks, 2024)](https://doi.org/10.5066/P1TLARC5).

In [ ]:
def calculate_percentiles(df, units) -> float:
    """
    Pre-processes streamflow data where there is an observed and modeled column. 
    Calculates percentiles for streamflow using Weibull plotting position for a fixed and variable threshold

    Returns
    -------
    dataframe
            |
    """
    # calculate day-of-year or julian day (important for variable threshold)
    # add it as a column to dataframe
    day_of_year = df.index.dayofyear
    df['jd'] = day_of_year

    # calculate climate year (important for NMAE drought signature evaluation)
    # add it as a column to dataframe
    df['cy'] = df.index.year.where(df.index.month < 4, df.index.year + 1)

    ## Handling of zeros
    # Zero flow cutoff (0.001 for cubic feet per second / 0.00028 for cubic meters per second)
    if units == 'cms':
        zero_cutoff = 0.00028
    elif units == 'cfs':
        zero_cutoff = 0.01

    # Create zero boolean columns (1 if present, 0 if not)
    df['zero_bool_obs'] = (df['observed'] < zero_cutoff).astype(int)
    df['zero_bool_mod'] = (df['modeled'] < zero_cutoff).astype(int)

    # Separate all zero vs not zero periods
    df['zero_id_obs'] = (df['zero_bool_obs'] != df['zero_bool_obs'].shift()).cumsum()
    df['zero_id_mod'] = (df['zero_bool_mod'] != df['zero_bool_mod'].shift()).cumsum()
    
    # Calculate zero stats for observations
    df_zero_obs = (df[df['zero_bool_obs'] == 1]
                   .groupby('zero_id_obs', as_index=False)
                   .apply(lambda x: x.assign(previous_zeros_obs=np.arange(1, len(x) + 1)), include_groups=False)
                   .reset_index(drop=True))
    df_zero_obs['time'] = df.index[df_zero_obs.index]

    # Calculate zero stats for modeled data
    df_zero_mod = (df[df['zero_bool_mod'] == 1]
                   .groupby('zero_id_mod', as_index=False)
                   .apply(lambda x: x.assign(previous_zeros_mod=np.arange(1, len(x) + 1)), include_groups=False)
                   .reset_index(drop=True))
    df_zero_mod['time'] = df.index[df_zero_mod.index]

    # Check if df_zero_obs and df_zero_mod have the expected columns
    has_previous_zeros_obs = 'previous_zeros_obs' in df_zero_obs.columns
    has_previous_zeros_mod = 'previous_zeros_mod' in df_zero_mod.columns

    # Prepare the DataFrames for merging
    df_zero_obs_selected = df_zero_obs[['time', 'previous_zeros_obs']] if has_previous_zeros_obs else df_zero_obs[['time']]
    df_zero_mod_selected = df_zero_mod[['time', 'previous_zeros_mod']] if has_previous_zeros_mod else df_zero_mod[['time']]

    # Attempt to join the two zero datasets together
    df_zero = pd.merge(df_zero_obs_selected, df_zero_mod_selected, on='time', how='outer')

    # If previous_zeros_obs or previous_zeros_mod is missing, fill with NaN
    if not has_previous_zeros_obs:
        df_zero['previous_zeros_obs'] = np.nan

    if not has_previous_zeros_mod:
        df_zero['previous_zeros_mod'] = np.nan

    # Combine zero information back with the main dataset for the gage
    gage_df = df.merge(df_zero, on=['time'], how='left')
    gage_df['previous_zeros_obs'] = gage_df['previous_zeros_obs'].fillna(0)
    gage_df['previous_zeros_mod'] = gage_df['previous_zeros_mod'].fillna(0)

    # originally multiplied by 0.01...
    gage_df['obs_prev_zeros'] = gage_df['observed'] - (gage_df['previous_zeros_obs'] * zero_cutoff)
    gage_df['mod_prev_zeros'] = gage_df['modeled'] - (gage_df['previous_zeros_mod'] * zero_cutoff)

    # sort values by time
    gage_df = gage_df.sort_values('time')

    # Compute the Weibull plotting position (r/(n + 1), where r is rank and n is the number of data (Lahaa et al,. 2017; https://doi.org/10.5194/hess-21-3001-2017)
    # no CDPM (continuous dry period method; https://doi.org/10.5194/hess-16-2437-2012)
    gage_df['weibull_site_obs_no_cdpm'] = 100 * gage_df['observed'].rank(method='min') / (gage_df['observed'].count() + 1)
    gage_df['weibull_site_mod_no_cdpm'] = 100 * gage_df['modeled'].rank(method='min') / (gage_df['modeled'].count() + 1)

    # CDPM (continuous dry period method; https://doi.org/10.5194/hess-16-2437-2012)
    gage_df['weibull_site_obs_cdpm'] = 100 * gage_df['obs_prev_zeros'].rank(method='min') / (gage_df['obs_prev_zeros'].count() + 1)
    gage_df['weibull_site_mod_cdpm'] = 100 * gage_df['mod_prev_zeros'].rank(method='min') / (gage_df['mod_prev_zeros'].count() + 1)
    
    # Assign a 'site' - this is to examine the percentiles over the period of record for the streamflow gage
    gage_df['site'] = gage_id

    # Group by site and day of year for Weibull calculations
    gage_df_grouped = gage_df.groupby(['site', 'jd'])

    # No CDPM
    gage_df['weibull_jd_obs_no_cdpm'] = gage_df_grouped['observed'].transform(lambda x: 100 * x.rank(method='min') / (x.count() + 1))
    gage_df['weibull_jd_mod_no_cdpm'] = gage_df_grouped['modeled'].transform(lambda x: 100 * x.rank(method='min') / (x.count() + 1))

    # CDPM
    gage_df['weibull_jd_obs_cdpm'] = gage_df_grouped['obs_prev_zeros'].transform(lambda x: 100 * x.rank(method='min') / (x.count() + 1))
    gage_df['weibull_jd_mod_cdpm'] = gage_df_grouped['mod_prev_zeros'].transform(lambda x: 100 * x.rank(method='min') / (x.count() + 1))

    # Choose cdpm or non cdpm
    gage_df['weibull_jd_obs'] = gage_df['weibull_jd_obs_cdpm']
    gage_df['weibull_jd_mod'] = gage_df['weibull_jd_mod_cdpm']
    gage_df['weibull_site_obs'] = gage_df['weibull_site_obs_cdpm']
    gage_df['weibull_site_mod'] = gage_df['weibull_site_mod_cdpm']

    gage_df['feature_id'] = gage_df['site']

    df_pct = gage_df
    
    return df_pct

## Calculate Drought Properties and Annual Statistics
Note: This function may have an issue with flow_volume. Bonus points if you find it.

This function is adapted from _calculate_properties_single.R_ in HyMED [(Simeone and Foks, 2024)](https://doi.org/10.5066/P1TLARC5).

In [ ]:
def calculate_site_properties(df, gage_id, 
                              thresholds = [5, 10, 20 , 30], 
                              percent_type_list = ['weibull_jd_obs', 'weibull_jd_mod', 'weibull_site_obs', 'weibull_site_mod'],
                              flow_name_list = ['observed', 'modeled', 'observed', 'modeled'], start_cy = 1985, end_cy = 2016) -> float:
    """
    This function identifies drought events and then calculates their properties.

    Returns
    -------
    dataframes
            |
    """
    low_flow_list = []
    drought_event_list = []
    flow_name = []
    
    df = df_pct
    
    # Calculate drought properties
    for h in range(len(percent_type_list)):
        percent_type = percent_type_list[h]
        flow_name = flow_name_list[h]
        df['flow_value'] = df[flow_name]

        # Iterate through each threshold.
        for j in range(len(thresholds)):
            thresh = thresholds[j]
        
            # Find periods under/over the threshold
            df['value'] = df[percent_type]
            df['less_than_thresh'] = np.where(df['value'] <= thresh, 1, 0) # create boolean 1 = under or at threshold (drought)
            df.sort_values(['time'], inplace = True)
            
            # Assign a unique identifier (drought_id) to consecutive periods of drought based on the less_than_thresh column
            df['drought_id'] = (df['less_than_thresh'] != df['less_than_thresh'].shift()).cumsum()
        
            # Group by 'feature_id' and the second part of percent_type, then calculate flow_thresh
            df['flow_thresh'] = df.groupby(['feature_id', percent_type.split('_')[1]])['flow_value'].transform(
                lambda x: np.quantile(x, thresh / 100, interpolation='linear'))
    
            # reset the index
            df.reset_index(drop=True, inplace=True)

            # Analysis on all periods below threshold
            df_low_flows = df[df['value'] <= thresh]
            df_low_flows = df_low_flows.groupby(['site', 'cy', 'drought_id']).agg(days=('value', 'size'),
                                                                          lowest_percent=('value', 'min'),
                                                                          severity=('value', lambda x: abs(np.sum(thresh - x))),
                                                                          lowest_flow=('flow_value', 'min'),
                                                                          flow_volume=('flow_thresh', lambda x: np.sum(x - df['flow_value']))).reset_index()

            # Summarize drought metrics
            drought_summary = (df_low_flows.groupby(['site', 'cy'])
                               .agg(total_duration_below_threshold=('days', 'sum'),
                                    longest_duration_below_threshold=('days', 'max'),
                                    total_severity_below_threshold=('severity', 'sum'),
                                    largest_severity_below_threshold=('severity', 'max'),
                                    maximum_drought_intensity=('lowest_percent', lambda x: thresh - x.min()),
                                    minimum_flow=('lowest_flow', 'min'),
                                    total_flow_volume_below_threshold=('flow_volume', 'sum'),
                                    largest_flow_volume_below_threshold=('flow_volume', 'max'))
                               .reset_index())

            # Complete the DataFrame to ensure all years are represented
            drought_summary = (drought_summary.set_index(['site', 'cy'])
                               .reindex(pd.MultiIndex.from_product([drought_summary['site'].unique(), range(start_cy, end_cy + 1)], names=['site', 'cy']),
                                        fill_value=0).reset_index())

            # Set 'minimum_flow' to NaN for the new rows (where the mask is False)
            existing_mask = drought_summary['minimum_flow'] != 0
            drought_summary['minimum_flow'] = np.where(existing_mask, drought_summary['minimum_flow'], np.nan)
    
            # Add additional columns
            drought_summary['threshold'] = thresh
            drought_summary['pct_type'] = percent_type
        

            # Analysis of drought events ------------------------------------------------
            # Amount of time between events for pooling and minimum time of event
            inter_event_duration = 5
            
            # Pooling severity ratio. The ratio of the inter-event excess severity to the preceding event deficit severity
            pooling_severity_ratio = 0.1
    
            # Finding events to be pooled.
            df_IC_events = df.groupby(['site', 'drought_id']).agg(duration=('value','count'),    # duration of each event
                                                                  drought_bool=('less_than_thresh', 'mean'),   # drought bool 1 is drought, 0 is not
                                                                  severity=('value', lambda x: abs((thresh - x).sum()))).reset_index()
            # Add columns with duration/severity of previous event
            df_IC_events['previous_duration'] = df_IC_events['duration'].shift(1)
            df_IC_events['previous_severity'] = df_IC_events['severity'].shift(1)
            df_IC_events['severity_ratio'] = df_IC_events['severity'] / df_IC_events['previous_severity']
            # Filter 
            df_IC_events = df_IC_events[(((df_IC_events['duration'] < inter_event_duration) & 
                                  (df_IC_events['previous_duration'] > df_IC_events['duration'])) | 
                                 (df_IC_events['severity_ratio'] < pooling_severity_ratio)) & 
                                 (df_IC_events['drought_bool'] == 0)]
            # Add feature id w/ event
            df_IC_events['feature_event_id'] = df_IC_events['site'] + "_" + df_IC_events['drought_id'].astype(str)

            # Run summary statistics on pooled droughts.
            df_drought_events = df[~df['drought_id'].isin(df_IC_events['drought_id'])].copy() # Remove inter-event periods that are too short/low severity to stop a drought.
            df_drought_events = df_drought_events[df_drought_events['less_than_thresh'] == 1] # Retain droughts
            df_drought_events = df_drought_events.groupby(['site', 'drought_id']).agg(
                    severity=('value', lambda x: (thresh - x).sum()),
                    mean_intensity=('value', 'mean'),
                    max_intensity=('value', 'min'),
                    duration=('value', 'count'),
                    start=('time', 'first'),
                    end=('time', 'last'),
                    min_flow=('flow_value', 'min'),
                    mean_flow=('flow_value', 'mean'),
                    max_flow=('flow_value', 'max'),
                    flow_volume=('flow_thresh', lambda x: (x - df.loc[x.index, 'flow_value']).sum(skipna=True))).reset_index()  # check this.. some flows are odd.

            # Keep drought events if duration is longer than or equal to the inter-event duration length in days
            df_drought_events = df_drought_events[df_drought_events['duration'] >= inter_event_duration]
    
            # Reassign climate year
            df_drought_events['cy'] = np.where(df_drought_events['start'].dt.month >= 4, df_drought_events['start'].dt.year + 1, df_drought_events['start'].dt.year)
    
            # Define the threshold and percentile
            df_drought_events['threshold'] = thresh
            df_drought_events['pct_type'] = percent_type
    
            # Append data
            low_flow_list.append(drought_summary)
            drought_event_list.append(df_drought_events)

    # Combine data for each threshold
    df_drought_events_all = pd.concat(drought_event_list, ignore_index=True)
    df_low_flows_all = pd.concat(low_flow_list, ignore_index=True)
    
    # Summarize data to the cy level
    df_drought_long = df_drought_events_all.groupby(['site', 'cy', 'threshold', 'pct_type']).agg(
                           total_drought_severity=('severity', 'sum'),
                           total_drought_duration=('duration', 'sum'),
                           number_of_drought_events=('severity', 'count'),
                           largest_drought_severity=('severity', 'max'),
                           longest_drought_duration=('duration', 'max'),
                           total_drought_flow_volume=('flow_volume', 'sum'),
                           longest_drought_flow_volume=('flow_volume', 'max')).reset_index()
                           #start_day_of_longest_drought=('start', lambda x: pd.to_datetime(x).dt.dayofyear.loc[x.idxmax()]),
                           #start_date_of_longest_drought=('start', lambda x: pd.to_datetime(x).loc[x.idxmax()]),
                           #end_date_of_longest_drought=('end', lambda x: pd.to_datetime(x).loc[x.idxmax()])
    
    # Ungroup and complete missing years (cy)
    start_cy = df_drought_long['cy'].min()
    end_cy = df_drought_long['cy'].max()
    
    # Create a complete DataFrame with all combinations of site, cy, threshold, and pct_type
    complete_index = pd.MultiIndex.from_product(
        [df_drought_long['site'].unique(), 
         range(start_cy, end_cy + 1), 
         df_drought_long['threshold'].unique(), 
         df_drought_long['pct_type'].unique()],
        names=['site', 'cy', 'threshold', 'pct_type'])

    # Reindex to complete the DataFrame
    df_drought_long = df_drought_long.set_index(['site', 'cy', 'threshold', 'pct_type']).reindex(complete_index, fill_value=0).reset_index()
    
    # Convert Drought event data to long format
    df_drought_long = df_drought_long.melt(id_vars=['site', 'cy', 'threshold', 'pct_type'],
                                           value_vars=['total_drought_severity', 
                                                       'total_drought_duration', 
                                                       'number_of_drought_events', 
                                                       'largest_drought_severity', 
                                                       'longest_drought_duration', 
                                                       'total_drought_flow_volume', 
                                                       'longest_drought_flow_volume'],
                                           var_name='measure',
                                           value_name='value')
    
    # Select the final columns
    df_drought_long = df_drought_long[['site', 'cy', 'measure', 'value', 'threshold', 'pct_type']]
    
    # Convert low flow threshold data to long format
    df_low_flow_long = df_low_flows_all.melt(
        id_vars=['site', 'cy', 'threshold', 'pct_type'], 
        value_vars=[
            'largest_flow_volume_below_threshold',
            'total_flow_volume_below_threshold',
            'minimum_flow',
            'total_severity_below_threshold',
            'largest_severity_below_threshold',
            'maximum_drought_intensity',
            'longest_duration_below_threshold',
            'total_duration_below_threshold'],  # Columns to unpivot
        var_name='measure',  # Name for the new variable column
        value_name='value' )  # Name for the new value column
    
    # Select the final columns (if needed)
    df_low_flow_long = df_low_flow_long[['site', 'cy', 'measure', 'value', 'threshold', 'pct_type']]
    
    # Combine datasets
    annual_stats = pd.concat([df_drought_long, df_low_flow_long], ignore_index=True)
    
    # Sort the combined DataFrame by 'site' and 'cy'
    annual_stats = annual_stats.sort_values(by=['site', 'cy'])

    # Remove duplicate rows
    annual_stats = annual_stats.drop_duplicates().reset_index(drop=True)

    return df_drought_events_all, annual_stats

## Simplified drought/non-drought boolean calculator
This function expands drought events into a TRUE/FALSE timeseries of drought/non-drought and is translated from the R script _calculate_site_booleans_threshold_only.R_ within HyMED (Simeone and Foks, 2024).

In [ ]:
def calculate_site_boolean_threshold_only(df_pct):
    """
    Expand drought events into true false timeseries of drought/no drought.
    We do this here only using thresholds not including pooling for improved simplicity of methodology.
    Returns
    -------
    dataframes
            |
    """

    # Subset dataframe
    df_bool_threshold_only = df_pct[['site', 'time', 'weibull_jd_obs', 'weibull_jd_mod', 'weibull_site_obs', 'weibull_site_mod']]
    
    # Pivot longer to get pct_type
    df_bool_threshold_only = df_bool_threshold_only.melt(id_vars=['site', 'time'], 
                                                               value_vars=['weibull_jd_obs', 'weibull_jd_mod', 'weibull_site_obs', 'weibull_site_mod'], 
                                                               var_name='pct_type',
                                                               value_name='value')
    #  Create boolean columns for each threshold                                                    
    df_bool_threshold_only['5'] = df_bool_threshold_only['value'] <= 5
    df_bool_threshold_only['10'] = df_bool_threshold_only['value'] <= 10
    df_bool_threshold_only['20'] = df_bool_threshold_only['value'] <= 20
    df_bool_threshold_only['30'] = df_bool_threshold_only['value'] <= 30                                        
    
    # Pivot longer for the thresholds
    df_bool_threshold_only = df_bool_threshold_only.melt(id_vars=['site', 'time', 'pct_type'], 
                                                         value_vars=['5', '10', '20', '30'], 
                                                         var_name='threshold', 
                                                         value_name='drought')
        
    # Convert threshold to numeric
    df_bool_threshold_only['threshold'] = pd.to_numeric(df_bool_threshold_only['threshold'])
        
    # rename 'time' to 'day'
    df_bool_threshold_only = df_bool_threshold_only.rename(columns={'time': 'day'})
        
    return df_bool_threshold_only

##  Event Classification: Cohen's kappa
Cohen's kappa [(Cohen, 1960)](https://doi.org/10.1177/001316446002000104) is used to measure the agreement between categorical variables. [Landis and Koch (1977)](https://doi.org/10.2307/2529310) provide a guide for interpretation. This script is translated from _site_cohens_kappas.R_ within HyMED (Simeone and Foks, 2024).

- Cohen, J. (1960). A coefficient of agreement for nominal scales. Educational and Psychological Measurement, 20, 37–46. https://doi.org/10.1177/001316446002000104
- Landis, J.R.; Koch, G.G. The measurement of observer agreement for categorical data. Biometrics 1977, 33, 159–174. https://doi.org/10.2307/2529310

In [ ]:
def site_cohens_kappa(df_bool, gage_id):
    
    # Setup lists to be filled.
    mod_ca_list = [np.nan] * 8
    mod_kappa_list = [np.nan] * 8
    pct_type_list = [np.nan] * 8
    threshold_list = [np.nan] * 8

    i = 0

    # Iterate through percent types.
    for percent_type in ["jd", "site"]:
        df_type = df_bool[df_bool['pct_type'].str.contains(percent_type)]

        # Iterate through percentiles.
        for thresh in [5, 10, 20, 30]:
            try:
                # Subset data to threshold value
                df = df_type[df_type['threshold'] == thresh]
                
                # Pivot the dataframe
                # Assign pct_type as a column and the drought False/True flag as the value.
                df = df.pivot_table(index=['threshold', 'day', 'site'], 
                                    columns='pct_type', 
                                    values='drought', 
                                    fill_value=False)
                df.reset_index(inplace=True)
                
                # Setup drought data
                df['drought_mod'] = df[f'weibull_{percent_type}_mod']
                df['drought_obs'] = df[f'weibull_{percent_type}_obs']
                df['mod_correct'] = df['drought_mod'] == df['drought_obs']

                # Calculate classification accuracy
                mod_ca_list[i] = df['mod_correct'].mean()

                # Calculate Cohen's kappa
                # Using scikit-learn
                # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html
                mod_kappa_list[i] = cohen_kappa_score(df['drought_mod'], df['drought_obs'])
                pct_type_list[i] = percent_type
                threshold_list[i] = thresh

            except Exception as e:
                print(f'Error encountered: {e}')
                mod_ca_list[i] = np.nan
                mod_kappa_list[i] = np.nan
                pct_type_list[i] = percent_type
                threshold_list[i] = thresh

            i += 1

    # Setup final DataFrame
    df_accuracy = pd.DataFrame({
        'site': gage_id,
        'threshold': threshold_list,
        'pct_type': pct_type_list,
        'mod_kappa': mod_kappa_list,
        'mod_classification_accuracy': mod_ca_list})

    return df_accuracy

### Error Components: Spearman's r
Spearman's rank correlation coefficient is calculated to evaluate a modeling applications' ability to reproduce the sequence of observational streamflow data (i.e., the timing of streamflow during streamflow drought periods). Spearman's assesses the monotonicity of the relation between the observed and simulated flows. This function is translated from _site_spearmans.R_ within HyMED (Simeone and Foks, 2024).

Observed and modeled streamflow are each first ranked by magnitude. After calculating ranks, we subset the data into periods with observed droughts at the various threshold levels of interest; for example, we took the lowest 20% of flow values for the 20th percentile drought threshold.

In [ ]:
def spearman_r(df_pct, site_name, thresholds = [5,10,20,30]):
    # Calculate Spearman's Rho
    
    # Setup df for use.
    df = df_pct.dropna(subset=['observed']).copy()
    df['obs_rank'] = df['observed'].rank(method='min')
    df['mod_rank'] = df['modeled'].rank(method='min')
    df['obs_mod_d'] = df['mod_rank'] - df['obs_rank']
    df['obs_mod_d2'] = df['obs_mod_d'] ** 2
    
    count = 0
    df_thresh_list = []
    result = {}

    # Iterate through thresholds.
    for thresh in thresholds:
        for pct_type in ['weibull_site', 'weibull_jd']:
            count += 1
        
            # Create the pct_value_obs column
            df['pct_value_obs'] = df[f'{pct_type}_obs']

            # Subset data to drought period and calculate drought metrics
            df_q = df[df['pct_value_obs'] <= thresh]

            # Calculate Spearman's Rho using scipy
            rho_obs_mod = (np.cov(df_q['obs_rank'], df_q['mod_rank'])) / (np.std(df_q['obs_rank']) * np.std(df_q['mod_rank']))
            rho_obs_mod = rho_obs_mod[0][1] # extract the correlation coefficient
            quant_length = len(df_q)

            # Create a result dictionary to store the results
            result = {
                'site': site_name,
                'pct_type': pct_type,
                'threshold': thresh,
                'quant_length' : quant_length,
                'spearmans_r': rho_obs_mod}
        
            # Append the result dictionary to results list
            df_thresh_list.append(result)
            
    # Concatenate all results into a single DataFrame
    df_spearmans = pd.DataFrame(df_thresh_list)

    return df_spearmans

### Bias & Ratio of Standard Deviations
Bias is a measure of the mean tendency of simulated values to be greater or less than associated observed values. Ratio of standard deviations (sd of modeled / sd of observed) is also calculated here. This function was translated from _site_bias_dist.R_ within HyMED (Simeone and Foks, 2024).

In [ ]:
def bias_dist(df_pct, site_name, thresholds = [5,10,20,30]):
    """
    Calculate bias and distributional errors.
    
    Parameters:
    df_pct (DataFrame): Input DataFrame containing 'observed' and 'modeled' values.
    site_name (str): Name of the site/gage of interest.
    thresholds (list): List of thresholds for calculations.

    Returns:
    DataFrame: A DataFrame containing calculated bias and distributional metrics.
    """
    # Initialize empty list
    df_thresh_list = []

    # Drop NA values from the 'q_obs' column
    df = df_pct.dropna(subset=['observed'])

    # Iterate through thresholds
    for thresh in thresholds:
        for pct_type in ['weibull_site', 'weibull_jd']:
            
            # Extract observed and modeled values
            df['pct_value_obs'] = df[f'{pct_type}_obs']
            df['pct_value_mod'] = df[f'{pct_type}_mod']
            
            # Filter data based on the threshold for observed values
            df_obs = df[df['pct_value_obs'] <= thresh]
            mean_obs = df_obs['observed'].mean()
            sd_obs = np.std(df_obs['observed'])

            # Filter data based on the threshold for modeled values
            df_mod = df[df['pct_value_mod'] <= thresh]
            mean_mod = df_mod['modeled'].mean()
            sd_mod = np.std(df_mod['modeled'])
            
            # Calculate metrics
            mod_sd_ratio = sd_mod / sd_obs if sd_obs != 0 else np.nan
            mod_bias = mean_mod - mean_obs
            mod_pct_bias = ((mean_mod - mean_obs) / abs(mean_obs)) * 100 if mean_obs != 0 else np.nan
            
            # Build DataFrame for return
            df_q = pd.DataFrame({
                'mod_bias': [mod_bias],
                'mod_pct_bias': [mod_pct_bias],
                'mod_sd_ratio': [mod_sd_ratio],
                'thresh': [thresh],
                'pct_type': [pct_type],
                'site': [site_name]
            })
            df_thresh_list.append(df_q)

    # Concatenate all results into a single DataFrame
    df_bias_sd = pd.concat(df_thresh_list, ignore_index=True)

    # Return DataFrame
    return df_bias_sd

### Error on Annual Signatures (NMAE and others)
The drought events that are identified each year are aggregated to determine the annual drought signatures of duration, intensity, and severity in each given climate year. The normalized mean absolute error (NMAE) on this annual time series is calculated to compare across streamgages and models. This is adapted from Simeone and Foks (2024), R package HyMED - script "site_annual_signature_metrics.R"

In [ ]:
def annual_signatures(df_annual_stats, site_name):
    """
    This function takes in a site and calculates metrics for various drought measures.

    Parameters:
    df_annual_stats (pd.DataFrame): The DataFrame containing annual statistics.
    site_name (str): The name of the site to filter the data.

    Returns:
    pd.DataFrame: A DataFrame containing the calculated metrics.
    """

    # Define the list of measures to consider
    measure_list = [
        "total_duration_below_threshold",
        "total_drought_severity",
        "total_flow_volume_below_threshold",
        "number_of_drought_events",
        "maximum_drought_intensity"]

    # Subset data
    df = df_annual_stats[df_annual_stats['measure'].isin(measure_list)]
    
    # Filter for the specific site
    df = df[df['site'] == site_name]

    # Replace NA values with 0
    df['value'] = df['value'].fillna(0)

    # Separate 'pct_type' into three columns: 'weibull', 'type', 'source'
    df[['weibull', 'type', 'source']] = df['pct_type'].str.split('_', expand=True)

    # Pivot wide
    df_wide = df.pivot_table(index=['site', 'measure', 'threshold', 'type'],
                              columns='source',
                              values='value',
                              fill_value=0).reset_index()

    # Filter thresholds
    df_wide = df_wide[df_wide['threshold'].isin([5, 10, 20, 30])]

    sum_df = df_wide.groupby(['site', 'measure', 'threshold', 'type']).agg(
        mae_mod=('mod', lambda x: np.mean(np.abs(df_wide.loc[x.index, 'obs'] - x))),
        mean_obs=('obs', lambda x: np.mean(x)),
        nmae_mod=('mod', lambda x: np.mean(np.abs(df_wide.loc[x.index, 'obs'] - x)) / np.mean(df_wide.loc[x.index, 'obs']))).reset_index()

    return sum_df